<a href="https://colab.research.google.com/github/seonmia/MRC_KoBigBird_KoELECTRA/blob/main/postprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 긴 길이의 예측값을 빈 문자열로 변화

#### long_string2blank() 함수

In [ ]:
import pandas as pd
def long_string2blank(predicted_dataset, max_length=10):
    """첫 번째 인수 : 예측 데이터셋<데이터프레임>. 단, 예측 데이터셋의 컬럼명에 'Predicted'가 존재해야 한다.
    두 번째 인수 : 허용 가능한 최대 길이<정수>
    이 함수가 돌아가기 위해선 import pandas as pd 를 선언해야 한다.
    """
    def convert2blank(row):
        predicted = row['Predicted']
        if len(predicted) > max_length:
            predicted = ""
        row['Predicted'] = predicted
        return row
    
    changed_dataset = predicted_dataset.apply(convert2blank, axis = 1)
    return changed_dataset

#### 사용 예시

In [ ]:
predicted_dataset = pd.read_csv('baseline.csv')
long_string2blank(predicted_dataset, 20)
predicted_dataset

,Id,Predicted
0,d14cb73158624cf094c546d856fd3c80,뉴
1,906631384e91493ebe1c7f34aea6f241,
2,35e61dcb479643448a2cb7d326ae50a6,미국
3,075e761b370040cb9041eecd39afc27c,“톱 50 가운데 22개가 IT 기업
4,e67ed38f3dd944be94d5b4c53731f334,백혈병
...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,200만엔
4004,cc7f826b66724ce9b39e3a974ca15661,
4005,3282034aa41e4fab980851ffd4a868dd,
4006,0a73550b36df4baf82ac2f98619d22e7,


## max_length의 적당한 값을 찾기
#### 훈련데이터셋의 정답레이블 길이정보를 분석

#### check_answers_length() 함수

In [ ]:
import pandas as pd
def check_answers_length(train_json):
    """인수로 json 훈련 데이터셋를 넣어준다.
    모든 answer들의 길이값과 텍스트를 추출하여 pandas의 DataFrame타입으로 반환한다.
    이 함수를 사용하려면 import pandas as pd 가 선언되어야 한다.
    """
    answers = []
    for sample in train_json:
        for answer in sample['paragraphs'][0]['qas'][0]['answers']:
            answers.append([len(answer['text']),answer['text']])
    answers_len_text = pd.DataFrame(answers, columns=['length', 'text'])
    return answers_len_text

#### 사용 예시

In [ ]:
import json
# 원본 파일을 로드한다.
with open('train.json', 'r', encoding='utf-8') as fd:
    data = json.load(fd)['data']

answers_len_text = check_answers_length(data)
# 반환값 확인
answers_len_text

,length,text
0,5,한 달가량
1,3,한 달
2,7,삼보테크놀로지
3,8,와쿠이 히데아키
4,9,‘교동반점 짬뽕’
...,...,...
14392,14,‘혹성탈출: 반격의 서막’
14393,12,혹성탈출: 반격의 서막
14394,6,8시 10분
14395,3,28개


#### answer들의 길이값 분석

In [ ]:
answers_len_text['length'].describe()

count    14397.000000
mean         6.040633
std          5.485509
min          1.000000
25%          3.000000
50%          5.000000
75%          7.000000
max        100.000000
Name: length, dtype: float64

길이의 평균값은 6, 최소값은 1, 최대값은 100이다.<br>
길이가 100짜리인 answer가 뭔지 확인해보자

In [ ]:
answers_len_text['text'][answers_len_text['length'].idxmax()]

'국제전기전자기술인협회 유럽 안테나 앤드 프로퍼게이션 컨퍼런스 (IEEE European Conference on Antennas and Propagation, EuCAP 2020)'

#### 전체 데이터 answer들의 75%는 7글자 이하이다.
#### 80%, 85%, 90%, 95%, 96%, 97%, 98%, 99%도 확인해보자

In [ ]:
answers_len_text['length'].describe([.75, .80, .85, .90, .95, .96, .97, .98, .99])

count    14397.000000
mean         6.040633
std          5.485509
min          1.000000
50%          5.000000
75%          7.000000
80%          8.000000
85%          9.000000
90%         11.000000
95%         15.000000
96%         17.000000
97%         19.000000
98%         22.000000
99%         28.000000
max        100.000000
Name: length, dtype: float64

위의 분석된 결과를 보고 적절한 max_length를 설정해주면 된다

## 의미가 있는 형태소만 추출

라이브러리 설치

In [ ]:
! apt-get install -y openjdk-8-jdk python3-dev
! pip install konlpy "tweepy<4.0.0"
! /bin/bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

#### pos_extraction() 메소드

In [ ]:
import pandas as pd
import konlpy

def pos_extraction(predicted_dataset):
    """첫 번째 인수 : 예측 데이터셋<데이터프레임>. 단, 예측 데이터셋의 컬럼명에 'Predicted'가 존재해야 한다.
    이 함수가 돌아가기 위해선 import pandas as pd 를 선언해야 한다.
    """
    tagger = konlpy.tag.Mecab()
    def select_pos(row):
        sub_morphs = set(['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VX', 'VCP', 'VCN', 'MM', 'MAG', 'MAJ', 'IC'])
        predicted = row['Predicted']
        result = ""
        if not(pd.isna(predicted)):                 # 예측답안이 NAN이 아닐때
            for word, tag in tagger.pos(predicted): # 예측답안을 형태소 분석
                if tag in sub_morphs:               # 의미가 있는 형태소이면
                    result += word                    # result에 추가한다.
        row['Predicted'] = result
        return row
    
    changed_dataset = predicted_dataset.apply(select_pos, axis = 1)
    return changed_dataset

#### 사용법

In [ ]:
predicted_dataset = pd.read_csv('baseline.csv')
pos_dataset = pos_extraction(predicted_dataset)
pos_dataset

,Id,Predicted
0,d14cb73158624cf094c546d856fd3c80,뉴
1,906631384e91493ebe1c7f34aea6f241,의도여객기하강버튼여객기하강속도사고직전조종석밖있조종사문여러차례두드리소리부조종사문열않설...
2,35e61dcb479643448a2cb7d326ae50a6,미국
3,075e761b370040cb9041eecd39afc27c,톱가운데개기업
4,e67ed38f3dd944be94d5b4c53731f334,백혈병
...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,만엔
4004,cc7f826b66724ce9b39e3a974ca15661,한국전쟁당시언니수지여동생오목이고의로방치수지신경쓰않오빠덕좋남편결혼그후여동생죄책감그녀수...
4005,3282034aa41e4fab980851ffd4a868dd,달러절반수준올들하락세계속있특히아시아유럽노선하락폭크이노선오가컨테이너선평균운임일달러주일전가량
4006,0a73550b36df4baf82ac2f98619d22e7,사흘간추이지켜보휴업더필요판단휴업연장여부포함별도조치마련서울교육청일서울전지역학교모든학생...


형태소 추출과 동시에 띄어쓰기 정보가 사라졌다.

## 띄어쓰기 정보 추가

라이브러리 설치

In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
import pandas as pd
from pykospacing import Spacing

def apply_spacing(predicted_dataset):
    """첫 번째 인수 : 예측 데이터셋<데이터프레임>. 단, 예측 데이터셋의 컬럼명에 'Predicted'가 존재해야 한다.
    이 함수가 돌아가기 위해선 import pandas as pd 를 선언해야 한다.
    """
    spacing = Spacing()
    def add_spacing(row):
        predicted = row['Predicted']
        row['Predicted'] = spacing(predicted) 
        return row
    
    changed_dataset = predicted_dataset.apply(add_spacing, axis = 1)
    return changed_dataset

In [ ]:
spacing_dataset = apply_spacing(pos_dataset)
spacing_dataset

,Id,Predicted
0,d14cb73158624cf094c546d856fd3c80,뉴
1,906631384e91493ebe1c7f34aea6f241,의도 여객기 하강 버튼 여객기 하강 속도사고 직전 조종석 밖 있조 종사문 여러 차례...
2,35e61dcb479643448a2cb7d326ae50a6,미국
3,075e761b370040cb9041eecd39afc27c,톱 가운데 개 기업
4,e67ed38f3dd944be94d5b4c53731f334,백혈병
...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,만엔
4004,cc7f826b66724ce9b39e3a974ca15661,한국전쟁 당시 언니 수지 여동생 오목이고의로 방치 수지신경 쓰 않오빠덕 좋남편 결혼...
4005,3282034aa41e4fab980851ffd4a868dd,달러 절반 수준 올들 하락 세계 속 있특히 아시아 유럽 노선 하락 폭 크 이 노선 ...
4006,0a73550b36df4baf82ac2f98619d22e7,사흘간 추이 지켜보 휴업 더 필요 판단 휴업 연장 여부 포함별도 조치 마련 서울교육...
